# Chapter 10: Building Workflows (Chaining)

**Scenario:** You write a draft, then you edit it. You don't do it all in one ink stroke. Similarly, complex AI tasks often require **Chaining**: using the output of one prompt as the input for the next.

In this finally chapter, you'll learn:
- ⛓️ **The Chain**: Passing data from Step 1 to Step 2.
- ↪️ **Self-Correction**: Asking the model to check its own work.
- **The Pipeline**: Building a multi-step workflow.

---

## 1. Setup

Standard setup.

In [1]:
!pip install -q litellm python-dotenv

import os
from dotenv import load_dotenv
from litellm import completion
import litellm
import logging

# Suppress noisy debug logs
litellm.suppress_debug_info = True
logging.getLogger("litellm").setLevel(logging.CRITICAL)

load_dotenv()
MODEL_NAME = os.getenv('DEFAULT_MODEL', 'gemini/gemini-2.5-flash')

In [2]:
def get_completion(prompt, system_prompt=None, temperature=0.0, max_tokens=1024):
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": prompt})
    
    response = completion(
        model=MODEL_NAME,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens
    )
    
    return response.choices[0].message.content

---

## 2. Self-Correction (The Double Check)

Sometimes models make mistakes. Paradoxically, if you ask them "Did you make a mistake?", they can often find it.

Let's try a tricky constraint: words ending in 'ab'.

In [3]:
prompt1 = "Name 10 words that end with the exact letters 'ab'."
response1 = get_completion(prompt1, temperature=0.7, max_tokens=2048)
print("--- Attempt 1 ---")
print(response1)

--- Attempt 1 ---
Okay, here are 10 words that end with "ab":

1.  **cab**
2.  **stab**
3.  **grab**
4.  **crab**
5.  **slab**
6.  **scab**
7.  **tab**
8.  **snob** (While not a traditional "ab" word, it technically ends in those letters)
9.  **bab** (A less common word, meaning a young child)
10. **Jab**



Now, we **chain** that response into a second prompt to verify it.

In [4]:
prompt2 = f"""
Here is a list of words generated by an AI:
{response1}

Please check each word. If it does not end in 'ab', provide a replacement word that does.
"""

response2 = get_completion(prompt2, temperature=0.0, max_tokens=2048)
print("--- Correction ---")
print(response2)

--- Correction ---
Okay, here's the corrected list:

1.  **cab** - Correct
2.  **stab** - Correct
3.  **grab** - Correct
4.  **crab** - Correct
5.  **slab** - Correct
6.  **scab** - Correct
7.  **tab** - Correct
8.  **snob** - Incorrect. Replacement: **fab** (short for fabulous)
9.  **bab** - Correct
10. **Jab** - Correct



## 3. The Extraction Pipeline

A common pattern in engineering is **Extract -> Process -> Format**.

**Task:** Extract names from a messy email, then alphabetize them.

**Step 1:** Extraction

In [5]:
email = """
Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. 
Keisha said she would come and I think Mel will be there too.
"""

prompt_step1 = f"""
Extract all names from the text below. Return them as a simple comma-separated list.

<text>
{email}
</text>
"""

names_list = get_completion(prompt_step1)
print(f"Extracted: {names_list}")

Extracted: Jesse, Erin, Joey, Keisha, Mel



**Step 2:** Transformation (Alphabetizing)

In [6]:
prompt_step2 = f"""
Alphabetize this list of names:
{names_list}
"""

final_output = get_completion(prompt_step2)
print("--- Final Output ---")
print(final_output)

--- Final Output ---
Here's the alphabetized list of names:

1.  Erin
2.  Jesse
3.  Joey
4.  Keisha
5.  Mel



---

## 4. Exercise: The Story Editor

**Goal:** Build a 3-step pipeline.
1.  **Draft**: Write a very short story (1 sentence) about a cat.
2.  **Critique**: Ask the model to critique the story's style.
3.  **Refine**: Rewrite the story based on the critique.

We've started the code for you. chaining the variables together.

In [7]:
# Step 1: Draft
prompt1 = "Write a one-sentence story about a cat who wants to fly."
draft = get_completion(prompt1, temperature=0.7, max_tokens=2048)
print(f"Draft: {draft}\n")

# Step 2: Critique
prompt2 = f"Critique this story for creativity and drama: '{draft}'"
critique = get_completion(prompt2, max_tokens=1000)
print(f"Critique: {critique}\n")

# Step 3: Refine
prompt3 = f"Rewrite the story '{draft}' to be more dramatic and 5 years old kid's friendly narrative, based on this critique: {critique}"
final_story = get_completion(prompt3, max_tokens=1000)

print("--- Final Story ---")
print(final_story)

Draft: Whiskers twitched with longing as Clementine the cat watched a robin soar, dreaming of trading her earthly paws for feathered wings and a life among the clouds.




Critique: Okay, let's break down the creativity and drama in this short story snippet:

**Creativity:**

*   **Basic Premise:** The core idea of a cat wanting to fly is a classic and relatable one. It taps into the universal desire for something beyond our current limitations. However, because it's so common, it needs a fresh angle to truly stand out.
*   **Imagery:** "Whiskers twitched with longing" is a good, evocative detail. It immediately puts us in the cat's perspective and hints at her inner desire. "Trading her earthly paws for feathered wings" is a clear and understandable metaphor. "Life among the clouds" is a standard but effective image of freedom and escape.
*   **Originality:** As it stands, the concept is not particularly original. Many stories explore the theme of wanting to be something else. To boost creativity, consider:
    *   **Unique Reason for Wanting to Fly:** Why *specifically* does Clementine want to fly? Is she escaping something? Is she driven by a specific

--- Final Story ---
Okay, here's a super fun and dramatic version of the story, perfect for a 5-year-old, based on your awesome critique!

**Clementine's Big Dream!**

Clementine the kitty cat wiggled her nose! *Wiggle, wiggle!* She saw a robin, a bright red birdy, flying WAY up high! Clementine's whiskers went *twitch, twitch, twitch!* Not because she was hungry, but because she had a BIG, SECRET DREAM!

Clementine didn't just want to chase string! She didn't just want yummy tuna! She wanted to FLY! She wanted to trade her little kitty paws for super-duper, sparkly wings!

Why? Because her bestest friend, Barnaby the bunny, was stuck! He was way up high in Farmer McGregor's tallest tree! (Uh oh!) Barnaby was too scared to come down!

Clementine knew if she had wings, she could zoom up, grab Barnaby, and fly him back down to safety! She imagined swooping through the fluffy clouds, feeling the wind tickle her fur! It would be the BEST!

But… how could a kitty cat get wings? Clementine p

---

## Conclusion

You have completed the **Practical Prompt Engineering** workshop! 🎉

You've learned:
1.  **Basic Structure**: Roles, Context, Instructions.
2.  **Clarity**: Cutting fluff, XML tags.
3.  **Formatting**: Outputting JSON/XML.
4.  **Reasoning**: Chain of Thought and Evidence.
5.  **Workflows**: Chaining multiple prompts.

Now go forth and build amazing AI applications!